# I. Import Needed Libraries

Selenium: accesses and controls web browser to submit search and locate web elements by xpath

CSV: reads and writes CSV files

re: regular expression for pattern matching

requests: makes HTTP requests

BeautifulSoup:parses HTML

pandas: data analysis library to manage dataframes


In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import csv, re, requests
from bs4 import BeautifulSoup
import pandas as pd

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed',)).History will not be written to the database.


# II. Instantiate Browser Object 

In [2]:
url = 'https://ww2.amstat.org/meetings/jsm/2009/onlineprogram/'

# chromedriver needs to be in exectuable path or you need to map to where your chromedriver is
browser = webdriver.Chrome()
browser.get(url)


# III. Navigate through webpage and scrape objects

### Creat xpaths for needed items

In [24]:
xpath_search = '/html/body/table[2]/tbody/tr[2]/td/form/div/input[6]'
xpath_session = '//tr//td[1]//a[contains(@href, "activity_details")]'
xpath_titles = '//tr//td[2]'
xpath_sponsors = '//tbody//tr//td[3]'
xpath_type = '//tbody//tr/td[4]'

### Click search with empty fields

In [21]:
# wouldn't click on this page so had to execute script instead of standard click() method
element = browser.find_element_by_xpath(xpath_search)
browser.execute_script("arguments[0].click();", element)


# browser.find_element_by_xpath(xpath_search).click()
# wait = WebDriverWait(browser, 30)
# wait.until(EC.presence_of_element_located((By.XPATH, xpath_search)))
# browser.find_element_by_xpath(xpath_search).click()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/table[2]/tbody/tr[2]/td/form/div/input[6]"}
  (Session info: chrome=63.0.3239.132)
  (Driver info: chromedriver=2.27.440174 (e97a722caafc2d3a8b807ee115bfb307f7d2cfd9),platform=Windows NT 6.1.7601 SP1 x86_64)


### Create lists of each field needed

In [46]:
sessions = [item.text for item in browser.find_elements_by_xpath(xpath_session) if 'Session' not in item.text]
titles = [item.text for item in browser.find_elements_by_xpath(xpath_titles) if item.text != "Title"]
sponsors = [item.text.replace('\n', ', ') for item in browser.find_elements_by_xpath(xpath_sponsors) 
            if item.text != 'Sponsor']
session_type = [re.sub(r'(\n|\r|\t)', ' ', item.text) for item in browser.find_elements_by_xpath(xpath_type) 
                if item.text != "Type" ]
session_linkToAbstract = [item.get_attribute('href') for item in browser.find_elements_by_xpath(xpath_session)]


In [47]:
len(sessions) == len(titles) == len(sponsors) == len(session_linkToAbstract)

True

In [55]:
df = pd.DataFrame({'Session_No' : sessions,
                   'Session_Title' : titles,
                   'Sponsors': sponsors,
                   'Type': session_type,
                   'Session_URL': session_linkToAbstract                   
                  })

In [56]:
df['Session_ID'] = df['Session_URL'].apply(lambda x: re.sub(r'https\S+((\d){6})', r'\1', x))

In [57]:
df.head(1)

,Session_No,Session_Title,Session_URL,Sponsors,Type,Session_ID
0,204907,Cancelled: George Mason University Alumni Rece...,https://ww2.amstat.org/meetings/jsm/2009/onlin...,George Mason University,Other ICW,204907


In [58]:
df.drop_duplicates(subset = 'Session_URL', inplace = True)
df.reset_index(inplace = True, drop= True)

In [59]:
dest = r'C:\Users\Thaunga\Scripts\02. Presentations\JSM\Datasets'
df.to_excel(r'{}\2009Sessions.xlsx'.format(dest))

# Test Code for this year's object locations (xpaths)

In [60]:
df[df['Session_No'].apply(lambda x: bool(re.search(r'(?<!\d)\d{1,3}(?!\d)',x)))]

,Session_No,Session_Title,Session_URL,Sponsors,Type,Session_ID
14,CE_01C,Analysis of Multivariate Survival Data,https://ww2.amstat.org/meetings/jsm/2009/onlin...,ASA,CE Course,204609
15,CE_02C,Missing Data in Longitudinal Studies: Strategi...,https://ww2.amstat.org/meetings/jsm/2009/onlin...,"ASA, Biometrics Section",CE Course,204611
16,CE_03C,Design of Experiments: New Methods and How to ...,https://ww2.amstat.org/meetings/jsm/2009/onlin...,"ASA, Section on Quality and Productivity",CE Course,204612
17,CE_04C,Longitudinal Data Analysis: Semiparametric and...,https://ww2.amstat.org/meetings/jsm/2009/onlin...,"ASA, Biometrics Section",CE Course,204613
18,CE_05C,The Analysis of Cross-Classified Categorical Data,https://ww2.amstat.org/meetings/jsm/2009/onlin...,"ASA, Section on Survey Research Methods",CE Course,204614
19,CE_06C,Applied Statistical Genetics for Population-Ba...,https://ww2.amstat.org/meetings/jsm/2009/onlin...,"ASA, Section on Statistics in Epidemiology",CE Course,204615
20,CE_07C,Active Control Clinical Trials,https://ww2.amstat.org/meetings/jsm/2009/onlin...,ASA,CE Course,204616
40,CE_01C,Analysis of Multivariate Survival Data,https://ww2.amstat.org/meetings/jsm/2009/onlin...,ASA,CE Course,204610
41,CE_08C,New Developments of Traditional Multivariate A...,https://ww2.amstat.org/meetings/jsm/2009/onlin...,ASA,CE Course,204617
42,CE_09C,Semiparametric Regression,https://ww2.amstat.org/meetings/jsm/2009/onlin...,ASA,CE Course,204618


In [132]:
test = df['Session_URL'].iloc[1]
test

'https://ww2.amstat.org/meetings/jsm/2009/onlineprogram/index.cfm?fuseaction=activity_details&sessionid=204843'

In [133]:
r = requests.get(test)
soup = BeautifulSoup(r.text, 'lxml')

# Run Code to extract workshops and abstract_urls

In [80]:
# dataframe of abstracts to scrape with new urls
session_url = []
abstract_url = []
abstract_title = []
session_no_v2 = []

#dataframe of abstracts already written in the session
workshop_abstract = []
workshop_session_no = []
workshop_session_url = []

for link in session_linkToAbstract:
    r = requests.get(link)
    soup = BeautifulSoup(r.text, 'lxml')
    
    # extract abstract titles and links
    abstract_links = [item.get('href') for item in soup.find_all('a') if re.search('abstract_details', str(item.get('href')))]
    
    if len(abstract_links) == 0:
        try:
            abstract_workshop = re.sub(r'(\n|\r|\t)', '', soup.find_all('tr')[9].text)
            id_no = soup.find_all('tr')[5].find('strong').text.strip()

            workshop_abstract.append(abstract_workshop)
            workshop_session_no.append(id_no)
            workshop_session_url.append(link)
        except:
            continue
    
    else:
        abstract_titles = [item.text for item in soup.find_all('a') 
            if re.search('abstract_details', str(item.get('href')))]

        # map back to session id and original link for later joining
        session_ids = [soup.find_all('tr')[5].find('strong').text] * len(abstract_links)
        original_link = [link] * len(abstract_links)

        # create list of 
        session_url = session_url + original_link
        abstract_url = abstract_url + abstract_links
        abstract_title = abstract_title + abstract_titles
        session_no_v2 = session_no_v2 + session_ids

    

In [81]:
len(session_url) == len(abstract_url) == len(abstract_title) == len(session_no_v2)

True

In [82]:
len(session_url)

2821

In [83]:
len(workshop_abstract) == len(workshop_session_no) == len(workshop_session_url)

True

In [84]:
len(workshop_abstract)

270

In [85]:
df_abstracts = pd.DataFrame({'Session_URL': session_url,
                            'Abstract_URL' : abstract_url,
                            'Abstract_Title': abstract_title,
                            'Session_No_v2': session_no_v2})

In [86]:
df_abstracts['Session_ID'] = df_abstracts['Session_URL'].apply(lambda x: re.sub(r'https\S+((\d){6})', r'\1', x))
df_abstracts['Abstract_URL'] = df_abstracts['Abstract_URL'].apply(lambda x: url + x)

In [87]:
df_abstracts.head(1)

,Abstract_Title,Abstract_URL,Session_No_v2,Session_URL,Session_ID
0,A Wavelet Analysis of the Role of Volatility i...,https://ww2.amstat.org/meetings/jsm/2009/onlin...,38,https://ww2.amstat.org/meetings/jsm/2009/onlin...,204558


In [88]:
df_workshops = pd.DataFrame({'Session_No': workshop_session_no,
                            'Abstract_Text': workshop_abstract,
                            'Session_URL': workshop_session_url})


In [89]:
df_workshops['Session_ID']= df_workshops['Session_URL'].apply(lambda x:re.sub(r'https\S+((\d){6})', r'\1', x))
df_workshops = df_workshops[df_workshops['Abstract_Text'].str.contains('Organizer\(s\)|Chair\(s\)') == False]

In [90]:
df_workshops.head(3)

,Abstract_Text,Session_No,Session_URL,Session_ID
0,,", -",https://ww2.amstat.org/meetings/jsm/2009/onlin...,204907
1,"JSM 2009For information, contact jsm@amstat.or...","Thu, 7/30/09, 7:00 PM - 9:00 PM",https://ww2.amstat.org/meetings/jsm/2009/onlin...,204843
2,"JSM 2009For information, contact jsm@amstat.or...","Fri, 7/31/09, 7:30 AM - 8:30 AM",https://ww2.amstat.org/meetings/jsm/2009/onlin...,204842


In [92]:
df_abstracts.drop_duplicates(subset = 'Abstract_URL', inplace = True)
df_abstracts.reset_index(drop = True, inplace = True)

In [97]:
df_abstracts.to_excel(r'{}\2009Abstracts.xlsx'.format(dest))
df_workshops.to_excel(r'{}\2009Workshops.xlsx'.format(dest))

# Test extraction of abstract info here

In [98]:
test = df_abstracts['Abstract_URL'].iloc[0]
test

'https://ww2.amstat.org/meetings/jsm/2009/onlineprogram/index.cfm?fuseaction=abstract_details&abstractid=303586'

In [99]:
r = requests.get(test)
soup = BeautifulSoup(r.text, 'lxml')

In [106]:
re.sub(r'\r|\n|\t', '', soup.find_all('tr')[14].find('p').text)

'Much recent work has documented and sought to explain declining macroeconomic volatility in the U.S. since the mid-1980s, known as The Great Moderation. One popular avenue of research has attempted to link this decline to advances in inventory management. In this paper we make use of wavelet statistical methods to further investigate this link. Using a specialized version of a discrete wavelet transform allows decomposition of the variance of a time series, as well as the covariance between series, on a scale by scale basis. We apply these methods to monthly data on aggregate inventories and sales in the manufacturing sector as well as the durable and nondurable goods subsectors. Initial results support findings of changes in volatility for sales and output growth taking place in the 1980s, but provide little evidence of reduced inventory growth volatilities playing a significant role. '

# Code to extract abstract details from abstract URLs

In [107]:
abs_num = []
abs_keyword = []
abs_type = []
abs_text = []

for link in df_abstracts['Abstract_URL']:
    r = requests.get(link)
    soup = BeautifulSoup(r.text, 'lxml')
    
    # abstract number
    try:
        number = re.sub(r'\S+abstractid=(\d+)', r'\1', link)
        abs_num.append(number)
    except:
        abs_num.append(None)
    
    # abstract keywords
    try:
        keywords = [item.text for item in soup.find_all('a') if re.search('keyword', str(item.get('href')))]
        keywords = ', '.join(keywords)
        abs_keyword.append(keywords)
    except:
        abs_keyword.append(None)
        
    
    # abstract type
    try:
        talk_type = re.sub(r'Type:|\n|\r|\t', '', soup.find_all('tr')[5].text)
        abs_type.append(talk_type)
    except:
        abs_type.append(None)
    
    # abstract text
    try:
        abstract_text = re.sub(r'\r|\n|\t', '', soup.find_all('tr')[14].find('p').text)
        abs_text.append(abstract_text)
    except:
        abs_text.append(None)
    

In [108]:
df_abstracts['Abstract_ID'] = abs_num
df_abstracts['Abstract_Keywords'] = abs_keyword
df_abstracts['Abstract_Type'] = abs_type
df_abstracts['Abstract_Text'] = abs_text

In [109]:
df_abstracts.to_excel(r'{}\2009Abstracts.xlsx'.format(dest))

In [111]:
df_abstracts.head(1)

,Abstract_Title,Abstract_URL,Session_No_v2,Session_URL,Session_ID,Abstract_ID,Abstract_Keywords,Abstract_Type,Abstract_Text
0,A Wavelet Analysis of the Role of Volatility i...,https://ww2.amstat.org/meetings/jsm/2009/onlin...,38,https://ww2.amstat.org/meetings/jsm/2009/onlin...,204558,303586,"volatility, output growth, wavelet, inventorie...",Contributed,Much recent work has documented and sought to ...


In [112]:
df_details = df_workshops.append(df_abstracts)

In [113]:
df.head(0)

,Session_No,Session_Title,Session_URL,Sponsors,Type,Session_ID


In [114]:
df_final = df_details.merge(df[['Session_Title', 'Sponsors', 'Type', 'Session_ID']], on = 'Session_ID', how= 'outer')

In [115]:
df_final['Unique'] = df_final['Session_ID'].map(str) + df_final['Abstract_ID'].map(str)

In [116]:
df_final.drop_duplicates(subset= 'Unique', inplace = True)
df_final.reset_index(drop = True, inplace = True)

In [145]:
df_final['Session_No'] = df_final['Session_No'].apply(lambda x: x if bool(str(x).startswith('CE') or str(x)[0].isdigit()) else "")

In [151]:
df_final['Session_No_Keep'] = df_final['Session_No_v2'].combine_first(df_final['Session_No'])

In [154]:
del df_final['Session_No']
del df_final['Session_No_v2']

In [157]:
df_final.rename(columns = {'Session_No_Keep': 'Session_No'}, inplace=True)

In [158]:
new_dest = r'C:\Users\Thaunga\Scripts\02. Presentations\JSM\Final Data'
df_final.to_excel(r'{}\2009_JSM_data.xlsx'.format(new_dest))